In [90]:
import os
import http.client
import json 

token = os.getenv("TOKEN")

conn = http.client.HTTPSConnection("brapi.dev")

headers = { 'Authorization': f'Bearer {token}'}

conn.request("GET", "/api/quote/PETR4,BOVA11?range=5y&interval=1d", headers=headers)

res = conn.getresponse()

data = res.read()

# print(data.decode("utf-8"))

json_data = json.loads(data.decode("utf-8"))



In [ ]:
import requests

token = "5Mx1PEHajEQdC7XrwX2XP3"
url = "https://brapi.dev/api/quote/ITUB4,BOVA11"
response = requests.get(
    url,
    headers={"Authorization": f"Bearer {token}"},
    params={"range": "5y", "interval": "1d"}
)
# if response.status_code == 200:
#     data = response.json()
#     print(data['results'])
# else:
#     print(f"Erro: {response.status_code}")

[{'symbol': 'ITUB4', 'shortName': 'ITUB4', 'longName': 'Itau Unibanco Holding SA Pfd', 'currency': 'BRL', 'regularMarketPrice': 48.42, 'regularMarketDayHigh': 49.19, 'regularMarketDayLow': 48.19, 'regularMarketDayRange': '48.19 - 49.19', 'regularMarketChange': 0.11, 'regularMarketChangePercent': 0.23, 'regularMarketTime': '2026-02-10T23:04:42.000Z', 'marketCap': 511537466770, 'regularMarketVolume': 43939300, 'regularMarketPreviousClose': 48.43, 'regularMarketOpen': 48.31, 'fiftyTwoWeekRange': '28.21804054 - 49.19', 'fiftyTwoWeekLow': 28.21804054, 'fiftyTwoWeekHigh': 49.19, 'priceEarnings': 12.344168260038241, 'earningsPerShare': 4.1106696, 'logourl': 'https://icons.brapi.dev/icons/ITUB4.svg', 'usedInterval': '1d', 'usedRange': '5y', 'historicalDataPrice': [{'date': 1613001600, 'open': 2.54, 'high': 2.56, 'low': 2.5, 'close': 2.52, 'volume': 246440002, 'adjustedClose': 0.19}, {'date': 1613088000, 'open': 2.5, 'high': 2.55, 'low': 2.49, 'close': 2.52, 'volume': 273553066, 'adjustedClose'

In [69]:
import pandas as pd

df = pd.DataFrame(data['results'])

df_stock = pd.DataFrame(data['results'][0]['historicalDataPrice'])
df_bova = pd.DataFrame(data['results'][1]['historicalDataPrice'])

df_stock['date'] = pd.to_datetime(df_stock['date'], unit='s')
df_bova['date'] = pd.to_datetime(df_bova['date'], unit='s')

print(df_bova.head())
print(df_stock.head())


                 date    open    high     low   close   volume  adjustedClose
0 2021-02-11 13:00:00  114.73  115.80  114.05  114.90  8336548         114.90
1 2021-02-12 22:05:00  114.29  115.26  113.66  115.05  6637473         115.05
2 2021-02-17 16:00:00  114.53  116.01  114.34  115.86  5357700         115.86
3 2021-02-18 13:00:00  115.75  116.25  114.01  114.61  8332508         114.61
4 2021-02-19 13:00:00  113.98  114.75  113.35  113.82  7258898         113.82
        date  open  high   low  close     volume  adjustedClose
0 2021-02-11  2.54  2.56  2.50   2.52  246440002           0.19
1 2021-02-12  2.50  2.55  2.49   2.52  273553066           0.19
2 2021-02-17  2.53  2.57  2.51   2.56  271050339           0.19
3 2021-02-18  2.54  2.59  2.53   2.54  542600123           0.19
4 2021-02-19  2.54  2.56  2.49   2.51  296193124           0.19


In [76]:
df_stock['rs'] = df_stock['close']/df_bova['close']

df_stock['ema20_price'] = df_stock['close'].ewm(span=20, adjust=False).mean()
df_stock['ema50_price'] = df_stock['close'].ewm(span=50, adjust=False).mean()

df_stock['ema21_rs'] = df_stock['rs'].ewm(span=21, adjust=False).mean()

df_stock['retorno'] = df_stock['close'].pct_change()*100

df_stock['acima_media'] = (df_stock['rs'] > df_stock['ema21_rs']).shift(1)

df_stock['tendencia_alta'] = (df_stock['ema20_price'] > df_stock['ema50_price']).shift(1)

df_stock['retorno'] = df_stock.apply(
    lambda row: row['retorno'] if row['acima_media'] == True and row['tendencia_alta'] == True else 0, axis=1
)


In [84]:
df_stock[df_stock['retorno'] != 0].mean()

date              2023-09-15 20:52:45
open                        11.383735
high                        11.498227
low                         11.269031
close                       11.384208
volume               269260195.122931
adjustedClose                9.193735
rs                            0.08587
ema21_rs                     0.079504
acima_media                       1.0
retorno                      2.158626
ema20_price                 10.470029
ema50_price                  9.344531
tendencia_alta                    1.0
dtype: object